# Quantitative Momentum Screener

## Table of Contents

## Project Overview

Momentum investing refers to investing in assets that have increased in price the most.

This project aims to build a momentum trading strategy which selects the 50 stocks with the hioghest price momentum. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks (from the S&P500 Index)


## Libraries

In [1]:
import numpy as np
import pandas as pd
import requests
import math
from scipy import stats

## Import List of Stocks
A list of ticker symbols in the S&P 500

In [2]:
tickers = pd.read_csv("../data/sp_500_stocks.csv")

## Setting Up the API
Connecting to the IEX Cloud API. This is the data provider that we will be using throughout these projects.

In [3]:
IEX_CLOUD_API_TOKEN = 'Tpk_059b97af715d417d9f49f50b51b1c448'

In [4]:
def api_url(tick):
    return f'https://sandbox.iexapis.com/stable/stock/{tick}/stats?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url('AAPL'))
data

<Response [200]>

## Executing A Batch API Call & Building Our DataFrame

In [5]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(tickers['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))

Real-world quantitative investment firms differentiate between "high quality" and "low quality" momentum stocks:

High-quality momentum stocks show "slow and steady" outperformance over long periods of time
Low-quality momentum stocks might not show any momentum for a long time, and then surge upwards.
The reason why high-quality momentum stocks are preferred is because low-quality momentum can often be cause by short-term news that is unlikely to be repeated in the future (such as an FDA approval for a biotechnology company).

To identify high-quality momentum, we're going to build a strategy that selects stocks from the highest percentiles of:

1-month price returns
3-month price returns
6-month price returns
1-year price returns
Let's start by building our DataFrame. You'll notice that I use the abbreviation hqm often. It stands for high-quality momentum.

In [6]:
cols =  [
    'Ticker', 
    'Price', 
    'Number of Shares to Buy', 
    'One-Year Price Return', 
    'One-Year Return Percentile',
    'Six-Month Price Return',
    'Six-Month Return Percentile',
    'Three-Month Price Return',
    'Three-Month Return Percentile',
    'One-Month Price Return',
    'One-Month Return Percentile',
    'HQM Score'
]

df = pd.DataFrame(columns=cols)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=stats,quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        df = df.append(pd.Series(
            [
                symbol, 
                data[symbol]['quote']['latestPrice'],
                'N/A',
                data[symbol]['stats']['year1ChangePercent'],
                'N/A',
                data[symbol]['stats']['month6ChangePercent'],
                'N/A',
                data[symbol]['stats']['month3ChangePercent'],
                'N/A',
                data[symbol]['stats']['month1ChangePercent'],
                'N/A',
                'N/A'
            ],index = cols), ignore_index = True)

df.dropna(subset = ['One-Year Price Return',
 'Six-Month Price Return',
 'Three-Month Price Return',
 'One-Month Price Return'], inplace=True)

df

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,147.24,N/A,0.224789,N/A,0.0036722,N/A,-0.0289657,N/A,-0.0144486,N/A,N/A
1,AAL,20.01,N/A,0.202913,N/A,-0.121516,N/A,-0.0979835,N/A,0.0371437,N/A,N/A
2,AAP,245.34,N/A,0.484487,N/A,0.170194,N/A,0.150307,N/A,0.0565745,N/A,N/A
3,AAPL,175.57,N/A,0.381844,N/A,0.220589,N/A,0.220763,N/A,0.0422136,N/A,N/A
4,ABBV,141.29,N/A,0.357893,N/A,0.20061,N/A,0.262923,N/A,0.114885,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,137.42,N/A,0.312172,N/A,0.191578,N/A,0.112156,N/A,0.0629722,N/A,N/A
501,ZBH,132.83,N/A,-0.196588,N/A,-0.209825,N/A,-0.122692,N/A,0.0285023,N/A,N/A
502,ZBRA,543.49,N/A,0.413815,N/A,0.0208338,N/A,0.099783,N/A,-0.0517398,N/A,N/A
503,ZION,72.82,N/A,0.475728,N/A,0.354258,N/A,0.108042,N/A,0.0801143,N/A,N/A


In [7]:
time_periods = ['One-Year','Six-Month','Three-Month','One-Month']

for row in df.index:
    for time_period in time_periods:
        df.loc[row, f'{time_period} Return Percentile'] = stats.percentileofscore(df[f'{time_period} Price Return'], df.loc[row, f'{time_period} Price Return'])/100

#Print the entire DataFrame    
df

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,147.24,N/A,0.224789,0.46507,0.0036722,0.289421,-0.0289657,0.157685,-0.0144486,0.167665,N/A
1,AAL,20.01,N/A,0.202913,0.429142,-0.121516,0.0878244,-0.0979835,0.0698603,0.0371437,0.421158,N/A
2,AAP,245.34,N/A,0.484487,0.826347,0.170194,0.724551,0.150307,0.766467,0.0565745,0.580838,N/A
3,AAPL,175.57,N/A,0.381844,0.696607,0.220589,0.832335,0.220763,0.912176,0.0422136,0.46507,N/A
4,ABBV,141.29,N/A,0.357893,0.672655,0.20061,0.788423,0.262923,0.954092,0.114885,0.9002,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,137.42,N/A,0.312172,0.606786,0.191578,0.770459,0.112156,0.634731,0.0629722,0.640719,N/A
501,ZBH,132.83,N/A,-0.196588,0.0239521,-0.209825,0.0319361,-0.122692,0.0439122,0.0285023,0.377246,N/A
502,ZBRA,543.49,N/A,0.413815,0.742515,0.0208338,0.325349,0.099783,0.608782,-0.0517398,0.0538922,N/A
503,ZION,72.82,N/A,0.475728,0.820359,0.354258,0.96008,0.108042,0.620758,0.0801143,0.740519,N/A


In [8]:
for row in df.index:
    df.loc[row, 'HQM Score'] = np.average([df.loc[row, f'{time_period} Return Percentile'] for time_period in time_periods])
    
df.sort_values(by = "HQM Score", inplace=True, ascending=False)
df = df[:51]
df = df.reset_index()

## Calculating the number of shares to buy

In [12]:
PORTFOLIO_SIZE = 10000000 #10 Mil 

In [15]:
position_size = float(PORTFOLIO_SIZE) / len(df.index)
for i in range(0, len(df['Ticker'])-1):
    df.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / list(df['Price'])[i])
df

,index,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,174,F,24.48,8009,1.86701,0.998004,0.712639,0.994012,0.762164,1,0.283667,1,0.998004
1,148,DVN,50.66,3870,1.81739,0.996008,0.813671,0.996008,0.283488,0.962076,0.16431,0.978044,0.983034
2,317,MRO,18.39,10662,1.4673,0.992016,0.385513,0.968064,0.201126,0.892216,0.156599,0.976048,0.957086
3,42,APA,29.87,6564,0.536723,0.866267,0.449372,0.988024,0.355271,0.982036,0.152488,0.974052,0.952595
4,175,FANG,122.34,1602,1.29855,0.988024,0.385497,0.966068,0.196071,0.874251,0.146893,0.968064,0.949102
5,37,ANET,132.08,1484,0.896268,0.978044,0.428735,0.982036,0.522135,0.998004,0.091982,0.800399,0.939621
6,50,AVGO,644.62,304,0.546472,0.868263,0.371698,0.964072,0.316988,0.97006,0.136926,0.9501,0.938124
7,89,CF,72.38,2709,0.677934,0.94012,0.406786,0.976048,0.167115,0.812375,0.17736,0.986028,0.928643
8,98,CMA,99.38,1973,0.653424,0.924152,0.432461,0.986028,0.166268,0.806387,0.165114,0.98004,0.924152
9,54,AZO,2037.23,96,0.707768,0.9501,0.344326,0.954092,0.250322,0.944112,0.098717,0.842315,0.922655
